In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import math
import random

In [2]:
# # Open the input file and output file in the same context
# with open('llmrawdata.txt', 'r') as infile, open('lmprocesseddata.txt', 'w') as outfile:
#     for line in infile:
#         if 'l4s_ecn_marking-start' in line:
#             # Remove the prefix 'l4s_ecn_marking-start,' and trailing ' end'
#             temp=line.split("-")
#             temp1=temp[1]
            
#             temp1=temp1.split(",")
#             #print(temp1[-1]=="end \n")
#             if(temp1[-1]=="end \n"):
#                 outfile.writelines(temp[1])


In [3]:
# Open the input file and output file in the same context
with open('llmrawdata.txt', 'r') as infile, open('lmprocesseddata.txt', 'w') as outfile:
    for line in infile:
        if 'l4s_ecn_marking-start' in line:
            # Remove the prefix 'l4s_ecn_marking-start,' and trailing ' end'
            parts = line.split("-")
            if len(parts) > 1:
                content = parts[1].strip()  # Remove any leading/trailing whitespace
                if content.endswith('end'):
                    # Write the content to the output file, stripping the 'end' part
                    outfile.write(content[6:-4] + '\n')


In [4]:
column_list = [
    "queue_type",                # q->queue_type
    "qdelay_reference",          # pprms->qdelay_ref
    "tupdate",               # pprms->tupdate
    "max_burst",            # pprms->max_burst
    "max_ecn_threshold",         # pprms->max_ecnth
    "alpha_coefficient",         # pprms->alpha
    "beta_coefficient",          # pprms->beta
    "flags",                     # pprms->flags
    "burst_allowance",           # pst->burst_allowance
    "drop_probability",          # pst->drop_prob
    "current_queue_delay",       # pst->current_qdelay
    "previous_queue_delay",      # pst->qdelay_old
    "accumulated_probability",   # pst->accu_prob
    "measurement_start_time",    # pst->measurement_start
    "average_dequeue_time",      # pst->avg_dq_time
    "dequeue_count",             # pst->dq_count
    "status_flags",              # pst->sflags
    "total_packets",             # q->stats.tot_pkts
    "total_bytes",               # q->stats.tot_bytes
    "queue_length",              # q->stats.length
    "length_in_bytes",           # q->stats.len_bytes
    "total_drops",               # q->stats.drops
    "dequeue_action",            # dequeue_action
]


In [5]:
df=pd.read_csv("lmprocesseddata.txt",names=column_list,header=None)

df["reward"] = df["current_queue_delay"]

In [6]:
df.describe()

,queue_type,qdelay_reference,tupdate,max_burst,max_ecn_threshold,alpha_coefficient,beta_coefficient,flags,burst_allowance,drop_probability,...,average_dequeue_time,dequeue_count,status_flags,total_packets,total_bytes,queue_length,length_in_bytes,total_drops,dequeue_action,reward
count,721858.000000,721858.0,721858.0,721858.0,721858.0,721858.0,721858.0,721858.0,721858.000000,7.218580e+05,...,721858.0,721858.0,721858.000000,721858.000000,7.218580e+05,721858.000000,721858.000000,721858.000000,721858.000000,721858.000000
mean,0.360528,15000.0,15000.0,150000.0,819.0,1024.0,10240.0,19.0,80466.913160,4.111521e+07,...,0.0,0.0,2.806986,21668.748211,1.942617e+07,40.351338,39755.218528,85.565251,1.005820,51253.376703
std,0.480154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64515.552538,1.216423e+08,...,0.0,0.0,0.736063,16982.194770,1.459151e+07,34.839060,37554.379666,87.202417,0.087483,48861.044328
min,0.000000,15000.0,15000.0,150000.0,819.0,1024.0,10240.0,19.0,0.000000,0.000000e+00,...,0.0,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,15000.0,15000.0,150000.0,819.0,1024.0,10240.0,19.0,0.000000,0.000000e+00,...,0.0,0.0,3.000000,7925.000000,7.336640e+06,14.000000,12185.000000,2.000000,1.000000,10000.000000
50%,0.000000,15000.0,15000.0,150000.0,819.0,1024.0,10240.0,19.0,105000.000000,3.276700e+04,...,0.0,0.0,3.000000,17916.000000,1.639200e+07,34.000000,31352.000000,66.000000,1.000000,40000.000000
75%,1.000000,15000.0,15000.0,150000.0,819.0,1024.0,10240.0,19.0,150000.000000,7.339371e+06,...,0.0,0.0,3.000000,31770.000000,2.878942e+07,57.000000,56800.000000,146.000000,1.000000,80000.000000
max,1.000000,15000.0,15000.0,150000.0,819.0,1024.0,10240.0,19.0,150000.000000,1.249636e+09,...,0.0,0.0,3.000000,80973.000000,7.019542e+07,323.000000,352384.000000,350.000000,3.000000,380000.000000


In [7]:

class ExperiencePool:
    """
    Experience pool for collecting trajectories.
    """
    def __init__(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.dones = []

    def add(self, state, action, reward, done):
        self.states.append(state)  # sometime state is also called obs (observation)
        self.actions.append(action)
        self.rewards.append(reward)
        self.dones.append(done)

    def __len__(self):
        return len(self.states)




import pickle
exp_pool = ExperiencePool()
# Initialize the global reward variable
global_reward = 0

# Iterate through each row and update the global reward variable
for index, row in df.iterrows():
    # global_reward += row['current_queue_delay']
    state = np.array(row[:])
    exp_pool.add(state=state, action=row['dequeue_action'], reward=row['current_queue_delay'], done=row['current_queue_delay'])



pickle_save_path='exp_pool.pkl'
pickle.dump(exp_pool, open( pickle_save_path, 'wb'))
print(f"Done. Experience pool saved at:", pickle_save_path)


Done. Experience pool saved at: exp_pool.pkl
